# Welcome to the COVID-19 Dashboard

This dashboard allows you to explore COVID-19 Cases, Hospital Admissions, Ventilator Bed Occupancy, data for England and Wales.

## Instructions:
1. Choose a Country: Select 'England' or 'Wales' from the dropdown.
2. Select Data Column: Choose a data column to plot from the dropdown (Cases, Hospital Admissions, or Ventilator Bed Occupancy).
3. Click 'Update Data' to refresh the COVID-19 data.
4. Click 'Plot Data' to generate the selected graph.

In [11]:
from uk_covid19 import Cov19API
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML
import json


%matplotlib inline
plt.rcParams['figure.dpi'] = 100

def fetch_data_for_nation(area_name):
    """Updates the Api of the specific area Name"""
    filters = [
        'areaType=nation',
        f'areaName={area_name}'
    ]

    structure = {
        'Cases': 'newCasesByPublishDate',
        'Hospital Admissions': 'newAdmissions',
        'Ventilator Bed Occupancy': 'covidOccupiedMVBeds',
        'date': 'date',
        'Area Name': 'areaName'
    }

    api = Cov19API(filters=filters, structure=structure)
    return api.get_json()

with open('hospitalEngland.json', 'r') as file:
    data_england = json.load(file)

with open('hospitalWales.json', 'r') as file:
    data_wales = json.load(file)


datalist_england = data_england['data']
datalist_wales = data_wales['data']

def parse_date(datestring):
    """Changes the format of the dates"""
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(datalist, area_name):
    """Wrangles the Data, Finds the min start date and then the end date, 
    and indexes them from the earliest date to the latest, and the frequency is set to daily.
    Then the a dataframe is created with the specific columns. Last the DataFrame is Populated"""
    dates = [parse_date(dictionary['date']) for dictionary in datalist]
    startdate = min(dates)
    enddate = max(dates)
    index = pd.date_range(startdate, enddate, freq='D')
    df = pd.DataFrame(index=index, columns=['Cases', 'Hospital Admissions', 'Ventilator Bed Occupancy'])

    for entry in datalist:
        date = parse_date(entry['date'])
        for column in ['Cases', 'Hospital Admissions', 'Ventilator Bed Occupancy']:
            value = 0.0 if entry.get(column) is None else float(entry[column])
            df.loc[date, column] = value

    df.fillna(0.0, inplace=True)
    return df

status_label = widgets.Label() #The status label widget

def update_dashboard(button):
    """The update api button. When it presses it will retrieve both,
    areas and update their apis. It will create the according json files 
    with orientation records. The wrangle function will be called, wrangle the data and create DataFrames. Then create a pickle. 
    status Loading Data will appear and as soon as
    everything is done either a successfull status will appear or an exception status will appear"""
    status_label.value = "Loading Data..."
    try:
        hospitalEngland = fetch_data_for_nation('England')
        hospitalWales = fetch_data_for_nation('Wales')

        df_england = wrangle_data(hospitalEngland['data'], 'England')
        df_wales = wrangle_data(hospitalWales['data'], 'Wales')

        df_england.to_json("hospitalsEngland.json", orient='records')
        df_wales.to_json("hospitalsWales.json", orient='records')
        df_england.to_pickle("hospitalsEngland.pkl")
        df_wales.to_pickle("hospitalsWales.pkl")

        status_label.value = "Data Loaded and Saved Successfully"
    except Exception as exe:
        status_label.value = f"Failed to load data: {exe}"

update_button = widgets.Button(description="Update Data", button_style='info', icon='exclamation')
update_button.on_click(update_dashboard)

output_widget = widgets.Output()

def load_data(country):
    """Whichever area is chosen the appropriate pickle file will be read"""
    if country == 'England':
        return pd.read_pickle('hospitalsEngland.pkl')
    elif country == 'Wales':
        return pd.read_pickle('hospitalsWales.pkl')

def update_plot(button):
    """An all value or a specific column will be chosen and the appropriate graph will be created, when the plot button will be pressed"""
    with output_widget:
        output_widget.clear_output()
        df = load_data(country_dropdown.value)
        column = column_dropdown.value

        plt.figure(figsize=(10, 6))
        if column == 'All':
            df.plot()
            plt.ylabel('Values')
        else:
            df[column].plot()
            plt.ylabel(column)

        plt.title(f'{column} Over Time in {country_dropdown.value}')
        plt.xlabel('Date')
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.show()


# explanation_text = widgets.HTML(
#     value="<h3>Welcome to the COVID-19 Dashboard</h3>"
#            "<p>This dashboard allows you to explore COVID-19 Cases, Hospital Admissions, Ventilator Bed Occupancy, data for England and Wales.</p>",
# )

# instructions_text = widgets.HTML(
#     value="<h4>Instructions:</h4>"
#            "<p>1. Choose a Country: Select 'England' or 'Wales' from the dropdown.</p>"
#            "<p>2. Select Data Column: Choose a data column to plot from the dropdown (Cases, Hospital Admissions, or Ventilator Bed Occupancy).</p>"
#            "<p>3. Click 'Update Data' to refresh the COVID-19 data.</p>"
#            "<p>4. Click 'Plot Data' to generate the selected graph.</p>",
# )

# display(explanation_text)
# display(instructions_text)

country_dropdown = widgets.Dropdown(options=['England', 'Wales'], description='Country:')
column_dropdown = widgets.Dropdown(options=['All', 'Cases', 'Hospital Admissions', 'Ventilator Bed Occupancy'], description='Column:')
plot_button = widgets.Button(description='Plot Data', button_style='success', icon='line-chart')

plot_button.on_click(update_plot)
display(country_dropdown, column_dropdown, update_button, plot_button,status_label, output_widget)

Dropdown(description='Country:', options=('England', 'Wales'), value='England')

Dropdown(description='Column:', options=('All', 'Cases', 'Hospital Admissions', 'Ventilator Bed Occupancy'), v…

Button(button_style='info', description='Update Data', icon='exclamation', style=ButtonStyle())

Button(button_style='success', description='Plot Data', icon='line-chart', style=ButtonStyle())

Label(value='')

Output()

**Author and Copyright Notice** Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) Copyright (C) Zinonas Mina 2023.